In [1]:
import os
import cv2

import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
dic_compteur_fruit = {}
i=0
for root, dir, filenames in os.walk('fruits-360_dataset/fruits-360/Training'):
    for filename in filenames:
        
        name = root.split("\\")[-1]
        n = name.split(" ")[0]
        if n in list(dic_compteur_fruit.keys()):
            dic_compteur_fruit[n] += 1
        else:
            dic_compteur_fruit[n] = 0
        old_path = root + "/" + filename
        newpath = "/".join(root.split("/")[0:-1]) + "/train/" + n + "/" + n + "_" + str(dic_compteur_fruit[n])+".jpg"
        os.rename(old_path,newpath) 

In [4]:
dic_compteur_fruit = {}
i=0
for root, dir, filenames in os.walk('fruits-360_dataset/fruits-360/Test'):
    for filename in filenames:
        
        name = root.split("\\")[-1]
        n = name.split(" ")[0]
        if n in list(dic_compteur_fruit.keys()):
            dic_compteur_fruit[n] += 1
        else:
            dic_compteur_fruit[n] = 0
        old_path = root + "/" + filename
        newpath = "/".join(root.split("/")[0:-1]) + "/testing/" + n + "/" + n + "_" + str(dic_compteur_fruit[n])+".jpg"
        os.rename(old_path,newpath) 

In [5]:
dic_compteur_fruit

{'Apple': 2133,
 'Apricot': 163,
 'Avocado': 308,
 'Banana': 483,
 'Beetroot': 149,
 'Blueberry': 153,
 'Cactus': 165,
 'Cantaloupe': 327,
 'Carambula': 165,
 'Cauliflower': 233,
 'Cherry': 1147,
 'Chestnut': 152,
 'Clementine': 165,
 'Cocos': 165,
 'Corn': 303,
 'Cucumber': 285,
 'Dates': 165,
 'Eggplant': 155,
 'Fig': 233,
 'Ginger': 98,
 'Granadilla': 165,
 'Grape': 1145,
 'Grapefruit': 329,
 'Guava': 165,
 'Hazelnut': 156,
 'Huckleberry': 165,
 'Kaki': 165,
 'Kiwi': 155,
 'Kohlrabi': 156,
 'Kumquats': 165,
 'Lemon': 329,
 'Limes': 165,
 'Lychee': 165,
 'Mandarine': 165,
 'Mango': 307,
 'Mangostan': 101,
 'Maracuja': 165,
 'Melon': 245,
 'Mulberry': 163,
 'Nectarine': 323,
 'Nut': 395,
 'Onion': 450,
 'Orange': 159,
 'Papaya': 163,
 'Passion': 165,
 'Peach': 573,
 'Pear': 1688,
 'Pepino': 165,
 'Pepper': 825,
 'Physalis': 327,
 'Pineapple': 328,
 'Pitahaya': 165,
 'Plum': 596,
 'Pomegranate': 163,
 'Pomelo': 152,
 'Potato': 600,
 'Quince': 165,
 'Rambutan': 163,
 'Raspberry': 165,
 

In [6]:
train_gen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range = 0.2,
    horizontal_flip=True,
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input,
    dtype=tf.float32
)

In [7]:
BATCH_SIZE = 256
IMG_SIZE = 75
TRAIN_DIR = 'fruits-360_dataset/fruits-360/train'
TEST_DIR = 'fruits-360_dataset/fruits-360/testing'

In [10]:
train_batch = train_gen.flow_from_directory(
    directory=TRAIN_DIR,
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="sparse",
    seed=1
)

Found 67692 images belonging to 67 classes.


In [11]:
test_batch = train_gen.flow_from_directory(
    directory=TEST_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode = 'sparse',
    seed=1
)

Found 22688 images belonging to 67 classes.


In [12]:
IMG_SHAPE = (IMG_SIZE,IMG_SIZE) + (3,)
base_model = tf.keras.applications.ResNet50(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

94781440/94765736 [==============================] - 55s 1us/step


In [14]:
base_model.trainable = False
inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(67, activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)

In [15]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)

In [17]:
h = model.fit(train_batch, 
            epochs=50,
            steps_per_epoch = 6231 // BATCH_SIZE,
            validation_data = test_batch,
            validation_steps = 3114 // BATCH_SIZE,
            batch_size=BATCH_SIZE,
            callbacks = tf.keras.callbacks.EarlyStopping(patience=2)
            )

Epoch 1/50
24/24 [==============================] - 38s 2s/step - loss: 0.6279 - accuracy: 0.8279 - val_loss: 0.5995 - val_accuracy: 0.8307
Epoch 2/50
24/24 [==============================] - 39s 2s/step - loss: 0.5377 - accuracy: 0.8416 - val_loss: 0.5248 - val_accuracy: 0.8581
Epoch 3/50
24/24 [==============================] - 40s 2s/step - loss: 0.4690 - accuracy: 0.8636 - val_loss: 0.4626 - val_accuracy: 0.8717
Epoch 4/50
24/24 [==============================] - 38s 2s/step - loss: 0.4130 - accuracy: 0.8818 - val_loss: 0.4433 - val_accuracy: 0.8809
Epoch 5/50
24/24 [==============================] - 39s 2s/step - loss: 0.3571 - accuracy: 0.8996 - val_loss: 0.3847 - val_accuracy: 0.8978
Epoch 6/50
24/24 [==============================] - 39s 2s/step - loss: 0.3280 - accuracy: 0.9015 - val_loss: 0.3712 - val_accuracy: 0.8952
Epoch 7/50
24/24 [==============================] - 38s 2s/step - loss: 0.2820 - accuracy: 0.9233 - val_loss: 0.3412 - val_accuracy: 0.8997
Epoch 8/50
24/24 [==